# Data Load

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import mean_absolute_error

from time import time

In [2]:

train = pd.read_csv('/Users/krc/TIL/Mini_PJT/data/train_V2.csv')

test = pd.read_csv('/Users/krc/TIL/Mini_PJT/data/test_V2.csv')

submission = pd.read_csv('/Users/krc/TIL/Mini_PJT/data/sample_submission_V2.csv')

# Data 전처리

In [3]:
# 불필요하다 판단되는 column 제거

df = train
df.drop(['Id', 'matchId', 'numGroups', 'killPlace'], axis=1, inplace=True)
df.columns

Index(['groupId', 'assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills',
       'heals', 'killPoints', 'kills', 'killStreaks', 'longestKill',
       'matchDuration', 'matchType', 'maxPlace', 'rankPoints', 'revives',
       'rideDistance', 'roadKills', 'swimDistance', 'teamKills',
       'vehicleDestroys', 'walkDistance', 'weaponsAcquired', 'winPoints',
       'winPlacePerc'],
      dtype='object')

In [4]:
# 결측치 값 1개로 삭제

df.dropna(axis=0, inplace=True)
df.isnull().sum()

groupId            0
assists            0
boosts             0
damageDealt        0
DBNOs              0
headshotKills      0
heals              0
killPoints         0
kills              0
killStreaks        0
longestKill        0
matchDuration      0
matchType          0
maxPlace           0
rankPoints         0
revives            0
rideDistance       0
roadKills          0
swimDistance       0
teamKills          0
vehicleDestroys    0
walkDistance       0
weaponsAcquired    0
winPoints          0
winPlacePerc       0
dtype: int64

# feature engineering

In [5]:


df['groupId'] = df['groupId'].astype('category')
df['groupId'] = df['groupId'].cat.codes

df['matchType'] = df['matchType'].astype('category')
df['matchType'] = df['matchType'].cat.codes


In [6]:
# df = numeric_dataset
df.isnull().sum()
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

fill_NaN = SimpleImputer(missing_values=np.nan, strategy='mean')
df_mu = pd.DataFrame(fill_NaN.fit_transform(df))
df.columns = df.columns
df.index = df.index
df.head()
df.columns

Index(['groupId', 'assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills',
       'heals', 'killPoints', 'kills', 'killStreaks', 'longestKill',
       'matchDuration', 'matchType', 'maxPlace', 'rankPoints', 'revives',
       'rideDistance', 'roadKills', 'swimDistance', 'teamKills',
       'vehicleDestroys', 'walkDistance', 'weaponsAcquired', 'winPoints',
       'winPlacePerc'],
      dtype='object')

In [7]:
train[['groupId', 'assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills',
       'heals', 'killPoints', 'kills', 'killStreaks', 'longestKill',
       'matchDuration', 'matchType', 'maxPlace', 'rankPoints', 'revives',
       'rideDistance', 'roadKills', 'swimDistance', 'teamKills',
       'vehicleDestroys', 'walkDistance', 'weaponsAcquired', 'winPoints',
       'winPlacePerc']] = df
       

In [8]:
# data leakage column, drop

from sklearn.model_selection import train_test_split

y = train['winPlacePerc']
X = train.drop(['winPlacePerc'], axis=1)
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.2, random_state=1511)

In [9]:

# 불필요한 램 용량 관리
import gc
gc.collect()

0

In [10]:

# 학습을 위한 라이브러리 세팅
from sklearn.linear_model import LinearRegression   # 1. Linear Regression
from sklearn.linear_model import Lasso              # 2. Lasso
from sklearn.linear_model import Ridge              # 3. Ridge
from xgboost.sklearn import XGBRegressor            # 4. XGBoost
from lightgbm.sklearn import LGBMRegressor          # 5. LightGBM

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

/Users/krc/miniforge3/envs/ml-modeling/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [11]:
## training
reg = LinearRegression()
reg2 = Lasso()
reg3 = Ridge()
reg4 = XGBRegressor()
reg5 = LGBMRegressor()

reg.fit(train_X, train_y)
reg2.fit(train_X, train_y)
reg3.fit(train_X, train_y)
reg4.fit(train_X, train_y)
reg5.fit(train_X, train_y)

pred_train = reg.predict(train_X)
pred_train2 = reg2.predict(train_X)
pred_train3 = reg3.predict(train_X)
pred_train4 = reg4.predict(train_X)
pred_train5 = reg5.predict(train_X)
pred_val = reg.predict(val_X)
pred_val2 = reg2.predict(val_X)
pred_val3 = reg3.predict(val_X)
pred_val4 = reg4.predict(val_X)
pred_val5 = reg5.predict(val_X)

mae_train = mean_absolute_error(train_y, pred_train)
mae_val = mean_absolute_error(val_y, pred_val)

mae_train2 = mean_absolute_error(train_y, pred_train2)
mae_val2 = mean_absolute_error(val_y, pred_val2)

mae_train3 = mean_absolute_error(train_y, pred_train3)
mae_val3 = mean_absolute_error(val_y, pred_val3)

mae_train4 = mean_absolute_error(train_y, pred_train4)
mae_val4 = mean_absolute_error(val_y, pred_val4)

mae_train5 = mean_absolute_error(train_y, pred_train5)
mae_val5 = mean_absolute_error(val_y, pred_val5)
# 결과


print("1. Linear Regression\t, train=%.4f, val=%.4f" % (mae_train, mae_val))
print("2. Lasso\t\t, train=%.4f, val=%.4f" % (mae_train2, mae_val2))
print("3. Ridge\t\t, train=%.4f, val=%.4f" % (mae_train3, mae_val3))
print("4. XGBoost\t\t, train=%.4f, val=%.4f" % (mae_train4, mae_val4))
print("5. LightGBM\t\t, train=%.4f, val=%.4f" % (mae_train5, mae_val5))

/Users/krc/miniforge3/envs/ml-modeling/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/krc/miniforge3/envs/ml-modeling/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/krc/miniforge3/envs/ml-modeling/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


1. Linear Regression	, train=0.1148, val=0.1149
2. Lasso		, train=0.1300, val=0.1303
3. Ridge		, train=0.1148, val=0.1149
4. XGBoost		, train=0.0894, val=0.0898
5. LightGBM		, train=0.0902, val=0.0904


In [12]:
from numpy import mean
from numpy import std
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

cv = KFold(n_splits=5, random_state=1511, shuffle=True)

scores = cross_val_score(reg, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
scores2 = cross_val_score(reg2, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
scores3 = cross_val_score(reg3, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
scores4 = cross_val_score(reg4, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
scores5 = cross_val_score(reg5, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

print('Accuracy: %.4f (%.4f)' % (mean(scores), std(scores)))
print('Accuracy: %.4f (%.4f)' % (mean(scores2), std(scores2)))
print('Accuracy: %.4f (%.4f)' % (mean(scores3), std(scores3)))
print('Accuracy: %.4f (%.4f)' % (mean(scores4), std(scores4)))
print('Accuracy: %.4f (%.4f)' % (mean(scores5), std(scores5)))

KeyboardInterrupt: 

- killplace drop
1. Linear Regression	, train=0.1147, val=0.1148
2. Lasso		, train=0.1300, val=0.1303
3. Ridge		, train=0.1147, val=0.1148
4. XGBoost		, train=0.0894, val=0.0898
5. LightGBM		, train=0.0902, val=0.0903

- groupId는 ordinal, matchId는 bynary Encoding 했을 경우,
1. Linear Regression	, train=0.0257, val=0.0258,	 train_a=0.1242, mae=0.1243
2. Lasso		, train=0.0300, val=0.0302,	 train_a=0.1373, mae=0.1376
3. Ridge		, train=0.0257, val=0.0258,	 train_a=0.1242, mae=0.1243
4. XGBoost		, train=0.0178, val=0.0180,	 train_a=0.0970, mae=0.0973

- groupId, matchId 둘다 ordinal_encoding 했을 경우,
1. Linear Regression	, train=0.0257, val=0.0258,	 train_a=0.1240, mae=0.1241
2. Lasso		, train=0.0300, val=0.0302,	 train_a=0.1373, mae=0.1376
3. Ridge		, train=0.0257, val=0.0258,	 train_a=0.1240, mae=0.1241
4. XGBoost		, train=0.0178, val=0.0180,	 train_a=0.0970, mae=0.0973

-- linear 와 lidge 가 mean_absolute_error 만 조금 좋아진걸 확인 할 수 있다.

- killPlace, numGroup(maxPlace 와 상관관계가 아주 높음(1)) Column drop
1. Linear Regression	, train=0.0232, val=0.0233,	 train_a=0.1148, mae=0.1149
2. Lasso		, train=0.0277, val=0.0278,	 train_a=0.1300, mae=0.1303
3. Ridge		, train=0.0232, val=0.0233,	 train_a=0.1148, mae=0.1149
4. XGBoost		, train=0.0157, val=0.0159,	 train_a=0.0894, mae=0.0898
5. LightGBM		, train=0.0160, val=0.0160,	 train_a=0.0902, mae=0.0904

- column no drop
1. Linear Regression	, train=0.0159, val=0.0159,	 train_a=0.0920, mae=0.0921
2. Lasso		, train=0.0241, val=0.0243,	 train_a=0.1209, mae=0.1211
3. Ridge		, train=0.0159, val=0.0159,	 train_a=0.0920, mae=0.0921
4. XGBoost		, train=0.0065, val=0.0067,	 train_a=0.0580, mae=0.0584
5. LightGBM		, train=0.0069, val=0.0069,	 train_a=0.0596, mae=0.0598

- 이상치를 제거
